In [2]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
import warnings
from sklearn.metrics import precision_score 
warnings.filterwarnings("ignore", category=DeprecationWarning)
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.layers import Dropout

Using Theano backend.


In [2]:
data = pd.read_csv("data.csv").fillna(0)
data

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5
5,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,9,3,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,6
6,Layup Shot,Layup,251,20000012,34.0443,0,0,-118.2698,8,3,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,7
7,Jump Shot,Jump Shot,254,20000012,34.0163,1,28,-118.2688,8,3,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,8
8,Jump Shot,Jump Shot,265,20000012,33.9363,-65,108,-118.3348,6,3,...,2PT Field Goal,Left Side(L),In The Paint (Non-RA),8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,9
9,Running Jump Shot,Jump Shot,294,20000012,33.9193,-33,125,-118.3028,3,3,...,2PT Field Goal,Center(C),In The Paint (Non-RA),8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,10


## Feature Transformation

In [3]:
shot_features_dict = dict([list(reversed(x)) for x in enumerate(set(data.action_type))])

shot_zone_basic_dict = dict([list(reversed(x)) for x in enumerate(set(data.shot_zone_basic))])

shot_type_dict = dict([list(reversed(x)) for x in enumerate(set(data.shot_type))])

shot_zone_range_dict = dict([list(reversed(x)) for x in enumerate(set(data.shot_zone_range))])

opponent_dict = dict([list(reversed(x)) for x in enumerate(set(data.opponent))])

year = [x.split("-")[0] for x in data.game_date]

action_type_num = [shot_features_dict.get(x) for x in data.action_type]

shot_zone_basic_num = [shot_zone_basic_dict.get(x) for x in data.shot_zone_basic]

shot_zone_range_num = [shot_zone_range_dict.get(x) for x in data.shot_zone_range]

opponent_num = [opponent_dict.get(x) for x in data.opponent]

shot_type_num = [shot_type_dict.get(x) for x in data.shot_type]

data['shot_zone_range_num'] = shot_zone_range_num

data['action_type_num'] = action_type_num

data['shot_zone_basic_num'] = shot_zone_basic_num

data['shot_type_num'] = shot_type_num

data['opponent_num'] = opponent_num

data['year'] = np.array(year)


In [4]:
possible_features = [u'action_type', u'combined_shot_type', u'game_event_id', u'game_id',
       u'lat', u'loc_x', u'loc_y', u'lon', u'minutes_remaining', u'period',
       u'playoffs', u'season', u'seconds_remaining', u'shot_distance',
       u'shot_made_flag', u'shot_type', u'shot_zone_area', u'shot_zone_basic',
       u'shot_zone_range', u'team_id', u'team_name', u'game_date', u'matchup',
       u'opponent', u'shot_id', 'shot_zone_basic', 'action_type_num', 'shot_zone_basic_num', 'year']

y = data[u'shot_made_flag']

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.25, random_state=42)

features = [
    u'shot_zone_range_num',
    u'year', 
    u'shot_type_num', 
    u'shot_distance', 
    u'seconds_remaining',  
    u'period', 
    u'minutes_remaining', 
    u'shot_zone_basic_num', 
    u'action_type_num',
    u'opponent_num']

X_train_features = X_train[features]

X_test_features = X_test[features]


## Logistic Regression

In [5]:
lr = LogisticRegression(n_jobs=-1)

lr.fit(X_train_features,y_train)

y_pred = [lr.predict([row]) for label,row in X_test_features.iterrows()]

precision_score(y_test, y_pred)

0.62307692307692308

## Neural Net

In [121]:
X_train_features.shape

(23022, 10)

In [123]:
y_train[1]

0.0

In [124]:
len(y_train)

23022

In [128]:
from collections import Counter
Counter(y_train)

Counter({0.0: 14480, 1.0: 8542})

In [10]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_train_binary = lb.fit_transform(y_train)

In [23]:
yb = []

for label in y_train:
    if label == 0:
        yb.append([0,1])
    else:
        yb.append([1,0])

In [18]:
from sklearn.preprocessing import OneHotEncoder

In [19]:
ohe = OneHotEncoder()
y_ohe = ohe.fit_transform(y_train)

In [22]:
y_ohe[0].toarray()

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [12]:
y_train_binary

array([[0],
       [0],
       [0],
       ..., 
       [0],
       [1],
       [0]])

In [17]:
a = [1.0,2.0,3.0,4.0]
lb.fit_transform(a)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [127]:
y_train_binary[0]

array([0])

In [5]:
x = [[0,1],[1,0],[0,0], [1,1]]
y = [1,1,0,0]

inputs = Input(shape=(2,))
layer = Dense(1, activation='sigmoid', kernel_initializer="uniform")(inputs)
model = Model(input=inputs, output=layer)
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(x, y, epochs=1000, validation_split=.15)
print model.predict(np.array([[1,1]]))
print model.predict(np.array([[1,0]]))
print model.predict(np.array([[0,1]]))
print model.predict(np.array([[0,0]]))

/Users/benjaminsmith/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=sigmoid.0, inputs=/input_3)`


Train on 3 samples, validate on 1 samples
Epoch 1/1000
3/3 [==============================] - 0s - loss: 0.6980 - val_loss: 0.6803
Epoch 2/1000
3/3 [==============================] - 0s - loss: 0.6975 - val_loss: 0.6818
Epoch 3/1000
3/3 [==============================] - 0s - loss: 0.6970 - val_loss: 0.6833
Epoch 4/1000
3/3 [==============================] - 0s - loss: 0.6965 - val_loss: 0.6848
Epoch 5/1000
3/3 [==============================] - 0s - loss: 0.6960 - val_loss: 0.6863
Epoch 6/1000
3/3 [==============================] - 0s - loss: 0.6955 - val_loss: 0.6878
Epoch 7/1000
3/3 [==============================] - 0s - loss: 0.6950 - val_loss: 0.6892
Epoch 8/1000
3/3 [==============================] - 0s - loss: 0.6945 - val_loss: 0.6907
Epoch 9/1000
3/3 [==============================] - 0s - loss: 0.6940 - val_loss: 0.6922
Epoch 10/1000
3/3 [==============================] - 0s - loss: 0.6935 - val_loss: 0.6937
Epoch 11/1000
3/3 [==============================] - 0s - loss: 0.6

In [37]:
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model

inputs = Input(shape=(10,))
hidden1 = Dense(2, activation='sigmoid', kernel_initializer="uniform")(inputs)
#hidden2 = Dense(2, activation='sigmoid', kernel_initializer="normal")(hidden1)
#predictions = Dense(1, kernel_initializer="normal")(hidden2)
model = Model(input=inputs, output=hidden1)
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X_train_features.as_matrix(), yb,
          epochs=10,
          validation_split=.15)

/Users/benjaminsmith/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=sigmoid.0, inputs=/input_14)`


Train on 19568 samples, validate on 3454 samples
Epoch 1/10
19568/19568 [==============================] - 0s - loss: 8.0590 - val_loss: 8.0590
Epoch 2/10
19568/19568 [==============================] - 0s - loss: 8.0590 - val_loss: 8.0590
Epoch 3/10
19568/19568 [==============================] - 0s - loss: 8.0590 - val_loss: 8.0590

In [7]:
from keras.utils.vis_utils import model_to_dot

In [8]:
from IPython.display import SVG
#from keras.utils.visualize_util import model_to_dot

SVG(model_to_dot(model, show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))

AttributeError: 'module' object has no attribute 'find_graphviz'

In [32]:

from keras.layers import Input
from keras.models import Model

inputs = Input(shape=(10,))
#hidden = Dense(10, activation='relu')(inputs)
predictions = Dense(2, activation='sigmoid')(inputs)

model = Model(input=inputs, output=predictions)

model.compile(loss='hinge',
              optimizer='adam')

model.fit(X_train_features, yb,
          nb_epoch=1,
          batch_size=500,
          validation_split=.15)

/Users/benjaminsmith/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=sigmoid.0, inputs=/input_9)`
/Users/benjaminsmith/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 19568 samples, validate on 3454 samples
Epoch 1/1


IndexError: indices are out-of-bounds

In [106]:
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model

inputs = Input(shape=(10,))
hidden1 = Dense(1, activation='sigmoid')(inputs)
#hidden2 = Dense(2, activation='sigmoid', kernel_initializer="normal")(hidden1)
#predictions = Dense(1, kernel_initializer="normal")(hidden2)
model = Model(input=inputs, output=hidden1)
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X_train_features.as_matrix(), y_train,
          epochs=10,
          validation_split=.15)

#y_pred = [model.predict(np.array(row).reshape(1, len(X_test_features.columns))) for label,row in X_test_features.iterrows()]

y_pred = model.predict(X_test_features.as_matrix())

#precision_score(y_test, y_pred)


/Users/benjaminsmith/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=sigmoid.0, inputs=dense_114/...)`


Epoch 1/10
23022/23022 [==============================] - 0s - loss: 5.9804     
Epoch 2/10
23022/23022 [==============================] - 0s - loss: 5.9804     
Epoch 3/10

In [38]:
y_pred = model.predict(X_test_features.as_matrix())


In [39]:
[x[0] for x in y_pred]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0